In [8]:
import nest_asyncio
nest_asyncio.apply()

In [21]:
from bs4 import BeautifulSoup as bs
import requests
import json
import os
from tqdm.asyncio import tqdm as atqdm
from tqdm import tqdm

import aiometer
import asyncio

In [10]:
SAVE_PATH = os.path.join(os.path.curdir, "data", "topical_guide", "scrape_tg.json")

In [11]:
BASE_URL = 'https://www.churchofjesuschrist.org/study/scriptures/tg'
LANGUAGE_QUERY = '?lang=eng'

In [12]:
topic_links = []

In [13]:
response = requests.get(BASE_URL + LANGUAGE_QUERY)
soup = bs(response.text, 'html.parser')
topic_links = soup.find_all('a', class_='item-U_5Ca')
topic_links = [BASE_URL + link['href'] for link in topic_links]

In [14]:
print(len(topic_links))
topic_links = topic_links[2:]
print(topic_links[0])

3514
https://www.churchofjesuschrist.org/study/scriptures/tg/study/scriptures/tg/aaron?lang=eng


In [17]:
topics_and_topic_links = []
for link in topic_links:
    topic = link.split('/')[-1]
    topic = topic.split('?')[0]

    topics_and_topic_links.append((topic, link))

print(topics_and_topic_links[0])

('aaron', 'https://www.churchofjesuschrist.org/study/scriptures/tg/study/scriptures/tg/aaron?lang=eng')


In [18]:
tg_data = {}
os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)

In [22]:
num_since_last_save = 0
for topic, link in tqdm(topics_and_topic_links):
    response = requests.get(link)
    soup = bs(response.text, 'html.parser')

    body_soup = soup.find('div', class_='body')
    if body_soup is None:
        print(f"Could not find body-block for {topic}")
        print(soup.text)
            

    p_tags = body_soup.find_all('p')
    dictionary_text = "\n".join([p.text for p in p_tags])

    tg_data[topic] = dictionary_text

    if num_since_last_save > 100:
        with open(SAVE_PATH, 'w+') as f:
            json.dump(tg_data, f)
        num_since_last_save = 0
    else:
        num_since_last_save += 1

 14%|█▍        | 492/3512 [21:18<2:10:47,  2.60s/it]


ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [ ]:


with open(SAVE_PATH, 'w+') as f:
    json.dump(formatted_data, f)

In [56]:
import asyncio
import httpx

In [66]:
async def fetch(url: str, topic:str, client: httpx.AsyncClient) -> None:
    try:
        response = await client.get(url)
        response.raise_for_status()

        soup = bs(response.text, 'html.parser')
        body_soup = soup.find('div', class_='body')
        if body_soup is None:
            print(f"Could not find body-block for {topic}")
            print(soup.text)
            

        p_tags = body_soup.find_all('p')
        dictionary_text = "\n".join([p.text for p in p_tags])

        return f"topic: {topic}\n{dictionary_text}"
    except httpx.HTTPError as e:
        print(f"An error occurred while fetching {url}: {e}")

    return ""

In [67]:
async def main(urls: list, topics:list,  max_connections: int = 15):
    async with httpx.AsyncClient(timeout=10.0, limits=httpx.Limits(max_connections=max_connections)) as client:
        progress_bar = atqdm(total=len(urls))
        async def wrapped_fetch(url, topic, client):
            result = await fetch(url, topic, client)
            progress_bar.update()
            return result

        # Wrap the tasks in wrapped_fetch
        tasks = [wrapped_fetch(url, topic, client) for url in urls]

        results = await asyncio.gather(*tasks)
        progress_bar.close()

    return results

In [68]:
topics = [topic for topic, _ in topics_and_topic_links]
urls = [url for _, url in topics_and_topic_links]
scraped_data = asyncio.run(main(urls, topics))

/home/dallin/miniconda3/envs/restoration_gpt/lib/python3.10/site-packages/anyio/_backends/_asyncio.py:524: RuntimeWarning: coroutine 'fetch' was never awaited
  cancel_scope = _task_states[task].cancel_scope


An error occurred while fetching https://www.churchofjesuschrist.org/study/scriptures/tg/study/scriptures/tg/black?lang=eng: 


An error occurred while fetching https://www.churchofjesuschrist.org/study/scriptures/tg/study/scriptures/tg/mystery?lang=eng: 


An error occurred while fetching https://www.churchofjesuschrist.org/study/scriptures/tg/study/scriptures/tg/skill?lang=eng: 
An error occurred while fetching https://www.churchofjesuschrist.org/study/scriptures/tg/study/scriptures/tg/skin?lang=eng: 
An error occurred while fetching https://www.churchofjesuschrist.org/study/scriptures/tg/study/scriptures/tg/israel-land-of?lang=eng: 
An error occurred while fetching https://www.churchofjesuschrist.org/study/scriptures/tg/study/scriptures/tg/skirt?lang=eng: 
An error occurred while fetching https://www.churchofjesuschrist.org/study/scriptures/tg/study/scriptures/tg/sky?lang=eng: 
An error occurred while fetching https://www.churchofjesuschrist.org/study/scriptures/tg/study/scriptures/tg/israel-origins-of?lang=eng: 
An error occurred while fetching https://www.churchofjesuschrist.org/study/scriptures/tg/study/scriptures/tg/slack?lang=eng: 
An error occurred while fetching https://www.churchofjesuschrist.org/study/scriptures/tg/study/scrip

An error occurred while fetching https://www.churchofjesuschrist.org/study/scriptures/tg/study/scriptures/tg/soap?lang=eng: 


100%|██████████| 3512/3512 [46:53<00:00,  1.25it/s]


In [69]:
formatted_data = {}
for i, data in enumerate(scraped_data):
    if data == "":
        print(f"Skipping {i} because it is empty")
        continue
    formatted_data[i] = data

Skipping 303 because it is empty
Skipping 1581 because it is empty
Skipping 1583 because it is empty
Skipping 2060 because it is empty
Skipping 2836 because it is empty
Skipping 2837 because it is empty
Skipping 2838 because it is empty
Skipping 2839 because it is empty
Skipping 2840 because it is empty
Skipping 2841 because it is empty
Skipping 2842 because it is empty
Skipping 2843 because it is empty
Skipping 2844 because it is empty
Skipping 2862 because it is empty


In [ ]:
os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)

with open(SAVE_PATH, 'w+') as f:
    json.dump(formatted_data, f)